# Scraping Google Jobs

Google jobs has their filters in the URL, so we simply need Pylenium to visit a URL to start scraping.

> This is actually pretty powerful now, so it's something I'd recommend people use!

Unlike other job sites, like Indeed, Google's results are _almost_ always relevant when it comes to jobs

In [1]:
from pylenium.driver import Pylenium, PyleniumConfig

from jobs import google

py = Pylenium(PyleniumConfig())
jobs = google.scrape(py, google.SOFTWARE_QA_ENGINEER_IN_UTAH)

In [2]:
from jobs import slack

# Google Jobs shares 20 jobs per page, so we don't need to limit this for Slack
payload = slack.create_google_jobs_payload(jobs)
response = slack.post_to_channel(payload)